<a href="https://colab.research.google.com/github/Imen-Khelfa/MOCA-Algorithm1/blob/main/MOCA_Algorithm_Application_on_CIFAR_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time
from datetime import datetime
from tensorflow.keras  import datasets
import matplotlib.pyplot as plt



Loading CIFAR-10 DATASET

In [ ]:
# loading CIFAR-10

(x_main, y_main) , (x_test, y_test)= datasets.cifar10.load_data()
x_main.shape

170498071/170498071 [==============================] - 4s 0us/step


(50000, 32, 32, 3)

Preprocessing Data:
- Splitting Data into Train, validation and test sets
- Data Normalization
- Encoding Labels

In [ ]:
#Splitting Data
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_main, y_main, test_size=0.2, random_state=42)


In [ ]:

# Normalization of images
x_train = x_train/255
x_val = x_val/255
x_test = x_test /255

In [ ]:
# Encoding of labels
y_train_cat = keras.utils.to_categorical(y_train, num_classes=10, dtype='float32')
y_val_cat = keras.utils.to_categorical(y_val, num_classes=10, dtype='float32')
y_test_cat = keras.utils.to_categorical(y_test, num_classes=10, dtype='float32')

In [ ]:
print(x_train.shape)
print(y_train_cat.shape)
print(x_val.shape)
print(x_test.shape)
print(y_test_cat.shape)


(40000, 32, 32, 3)
(40000, 10)
(10000, 32, 32, 3)
(10000, 32, 32, 3)
(10000, 10)


**MOCA**

In [ ]:
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, Flatten, Dense
from tensorflow.keras.callbacks import TensorBoard
import datetime


In [ ]:
log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
def get_memory_footprint(model):
    model_bytes = model.count_params() * 4  # Assume 32-bit float size
    return model_bytes / 1024 / 1024  # Convert to megabytes

In [ ]:

def get_inference_time(model, X_test):
    start_time = time.time()
    model.predict(X_test)
    end_time = time.time()
    return end_time - start_time

In [ ]:
def calculate_fitness(accuracy, inference_time, memory_footprint):
    # Define weights for each metric
    weight_accuracy = 1
    weight_inference_time = -1
    weight_memory_footprint = -1

    # Calculate fitness as weighted sum of metrics
    fitness = weight_accuracy * accuracy + weight_inference_time * inference_time + weight_memory_footprint * memory_footprint

    return fitness

In [ ]:
def rank_models(models):
    ranked_models = sorted(models[0], key=lambda x: x['fitness'], reverse=True)
    return ranked_models

In [ ]:
import random

def generate_random_architecture():
       architecture_type = 'cnn'  # You can switch between 'cnn' and 'rnn'
       input_shape = (32, 32, 3)
       num_classes = 10
       activation_functions = ['relu', 'sigmoid']
       optimizers = ['adam', 'rmsprop']

       models = []

       if architecture_type == 'cnn':
            # Generate a random CNN architecture
            num_models = random.randint(1, 3)  # Random number of models

            for _ in range(num_models):
                model_layers = []
                num_layers = random.randint(1, 3)  # Limit the number of convolutional layers

                for _ in range(num_layers):
                    filters = random.randint(16, 128)
                    kernel_size = random.choice([(3, 3), (5, 5)])
                    activation = random.choice(activation_functions)

                    # Check if the layer is valid for the current input shape
                    if input_shape[0] >= kernel_size[0] and input_shape[1] >= kernel_size[1]:
                        layer = {
                            'type': 'Conv2D',
                            'filters': filters,
                            'kernel_size': kernel_size,
                            'activation': activation
                        }

                        model_layers.append(layer)

                        # Update the input shape after Conv2D
                        input_shape = (input_shape[0] - kernel_size[0] + 1, input_shape[1] - kernel_size[1] + 1, filters)

                        # Add a MaxPooling layer to reduce dimensions (e.g., 2x2)
                        if input_shape[0] >= 2 and input_shape[1] >= 2:
                            model_layers.append({
                                'type': 'MaxPooling2D',
                                'pool_size': (2, 2)
                            })

                if not model_layers:
                    # Ensure at least one Conv2D layer is added
                    filters = random.randint(16, 128)
                    kernel_size = random.choice([(3, 3), (5, 5)])
                    activation = random.choice(activation_functions)

                    layer = {
                        'type': 'Conv2D',
                        'filters': filters,
                        'kernel_size': kernel_size,
                        'activation': activation
                    }

                    model_layers.append(layer)

                    # Add a MaxPooling layer to reduce dimensions (e.g., 2x2)
                    if input_shape[0] >= 2 and input_shape[1] >= 2:
                        model_layers.append({
                            'type': 'MaxPooling2D',
                            'pool_size': (2, 2)
                        })

                model_layers.append({
                    'type': 'Flatten'
                })

                model_layers.append({
                    'type': 'Dense',
                    'units': num_classes,
                    'activation': 'softmax'
                })

                model = {
                    'layers': model_layers,
                    'activation': random.choice(activation_functions),
                    'optimizer': random.choice(optimizers),
                    'accuracy': 0.0,
                    'inference_time': 0.0,
                    'f1score': 0.0,
                    'memory_footprint': 0.0,
                    'fitness': 0.0
                }

                models.append(model)

        # You can add a similar section for generating RNN architectures here

       return models

In [ ]:
import time
import tensorflow as tf
from sklearn.metrics import accuracy_score
import numpy as np

def evaluate_model(models, x_train, y_train_cat, x_val, y_val_cat, x_test, y_test_cat):
    for model in models:
        model_dict = model[0]  # Access the dictionary inside the list
        model_layers = model_dict['layers']  # Access the 'layers' key
        activation = model_dict['activation']
        optimizer = model_dict['optimizer']

        # Build the model using the layer configurations
        model_obj = tf.keras.Sequential()

        # Add an input layer with the correct input shape
        model_obj.add(tf.keras.layers.Input(shape=(32, 32, 3)))

        for layer in model_layers:
            layer_type = layer['type']

            if layer_type == 'Conv2D':
                filters = layer['filters']
                kernel_size = layer['kernel_size']

                model_obj.add(tf.keras.layers.Conv2D(filters, kernel_size, activation=activation, padding='same'))
                model_obj.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

            elif layer_type == 'MaxPooling2D':
                pool_size = layer['pool_size']

                model_obj.add(tf.keras.layers.MaxPooling2D(pool_size=pool_size))

            elif layer_type == 'Flatten':
                model_obj.add(tf.keras.layers.Flatten())

        model_obj.add(tf.keras.layers.Dense(10, activation='softmax'))

        # Compile the model
        model_obj.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

        # TensorBoard callback
        log_dir = f"./logs/{time.time()}"
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

        # Use preprocessed data for training
        model_obj.fit(x_train, y_train_cat, epochs=10, batch_size=32, validation_data=(x_val, y_val_cat), callbacks=[tensorboard_callback])

        # Evaluate the model
        y_pred = model_obj.predict(x_test)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_test_classes = np.argmax(y_test_cat, axis=1)  # Convert y_test_cat to class labels
        accuracy = accuracy_score(y_test_classes, y_pred_classes)
        inference_time = get_inference_time(model_obj, x_test)
        memory_footprint = get_memory_footprint(model_obj)

        # Calculate fitness function
        fitness = calculate_fitness(accuracy, inference_time, memory_footprint)

        # Assign the evaluation results to the model dictionary
        model_dict['accuracy'] = accuracy
        model_dict['inference_time'] = inference_time
        model_dict['memory_footprint'] = memory_footprint
        model_dict['fitness'] = fitness

    return models


In [ ]:
def perform_crossover(parent1, parent2):
    child = {}

    # Perform single-point crossover at a randomly selected index
    crossover_point = random.randint(0, len(parent1['layers']) - 1)

    # Combine the properties of parent1 and parent2 up to the crossover point
    child['layers'] = parent1['layers'][:crossover_point] + parent2['layers'][crossover_point:]
    child['activation'] = parent1['activation']
    child['optimizer'] = parent2['optimizer']

    return child

In [ ]:

def perform_mutation(model):
    mutated_model = copy.deepcopy(model)

    # Randomly select a hyperparameter to mutate
    hyperparameter = random.choice(['units', 'activation', 'optimizer'])

    if hyperparameter == 'units':
        # Mutate the number of units in a randomly selected layer
        layer_index = random.randint(0, len(mutated_model['layers']) - 1)
        layer = mutated_model['layers'][layer_index]
        new_units = random.randint(16, 512)  # Randomly select a new number of units
        layer['units'] = new_units

    elif hyperparameter == 'activation':
        # Mutate the activation function in a randomly selected layer
        layer_index = random.randint(0, len(mutated_model['layers']) - 1)
        layer = mutated_model['layers'][layer_index]
        activations = ['relu', 'sigmoid', 'tanh', 'softmax']
        new_activation = random.choice(activations)  # Randomly select a new activation function
        layer['activation'] = new_activation

    elif hyperparameter == 'optimizer':
        # Mutate the optimizer used in the model
        optimizers = ['adam', 'sgd', 'rmsprop']
        new_optimizer = random.choice(optimizers)  # Randomly select a new optimizer
        mutated_model['optimizer'] = new_optimizer

    return mutated_model

In [ ]:
def propagate_weights(parent, mutated_model):
    propagated_model = copy.deepcopy(mutated_model)

    # Copy the weights from the parent model to the mutated model
    for parent_layer, mutated_layer in zip(parent['layers'], propagated_model['layers']):
        # Assuming the layers have compatible shapes, copy the weights
        mutated_layer['weights'] = copy.deepcopy(parent_layer['weights'])
        mutated_layer['biases'] = copy.deepcopy(parent_layer['biases'])

    return propagated_model

In [ ]:
import numpy as np

def reduce_parameters(model):
    pruning_threshold = 0.2
    reduced_model = copy.deepcopy(model)

    # Iterate through the layers of the model
    for layer in reduced_model['layers']:
        weights = layer['weights']

        # Calculate the magnitude of each weight
        weight_magnitudes = np.abs(weights)

        # Identify the weights below the pruning threshold
        prune_mask = weight_magnitudes < pruning_threshold

        # Set the pruned weights to zero
        weights[prune_mask] = 0.0

    return reduced_model

In [ ]:
K = 22 # Population size
G = 10   # Number of generations
N = 10   # Number of top models to duplicate
C = 20   # Number of offspring to generate
threshold_accuracy = 0.9  # Accuracy threshold

In [ ]:
# Initialize the population with random CNN and RNN architectures
population = [generate_random_architecture() for _ in range(K)]

In [ ]:
population

[[{'layers': [{'type': 'Conv2D',
     'filters': 116,
     'kernel_size': (3, 3),
     'activation': 'sigmoid'},
    {'type': 'MaxPooling2D', 'pool_size': (2, 2)},
    {'type': 'Conv2D',
     'filters': 76,
     'kernel_size': (3, 3),
     'activation': 'sigmoid'},
    {'type': 'MaxPooling2D', 'pool_size': (2, 2)},
    {'type': 'Conv2D',
     'filters': 112,
     'kernel_size': (5, 5),
     'activation': 'relu'},
    {'type': 'MaxPooling2D', 'pool_size': (2, 2)},
    {'type': 'Flatten'},
    {'type': 'Dense', 'units': 10, 'activation': 'softmax'}],
   'activation': 'relu',
   'optimizer': 'adam',
   'accuracy': 0.0,
   'inference_time': 0.0,
   'f1score': 0.0,
   'memory_footprint': 0.0,
   'fitness': 0.0},
  {'layers': [{'type': 'Conv2D',
     'filters': 68,
     'kernel_size': (3, 3),
     'activation': 'relu'},
    {'type': 'MaxPooling2D', 'pool_size': (2, 2)},
    {'type': 'Conv2D',
     'filters': 92,
     'kernel_size': (5, 5),
     'activation': 'sigmoid'},
    {'type': 'MaxPool

In [ ]:
# Initialize lists to store the best hyperparameters and metrics
best_hyperparameters = []
accuracies = []
inference_times = []
memory_footprints = []

In [ ]:
import random
import copy


 # Run the algorithm for G generations
for generation in range(G):
    # Evaluate & rank the models
    models = evaluate_model(population, x_train, y_train_cat, x_val, y_val_cat, x_test, y_test_cat)
    ranked_models = rank_models(models)

    # Select the top N models for duplication
    selected_models = ranked_models[:N]

    # Duplicate the selected models
    duplicated_models = copy.deepcopy(selected_models)

    # Generate C offspring through crossover
    offspring = []
    for _ in range(C):
        parent1, parent2 = random.sample(selected_models, 2)
        child = perform_crossover(parent1, parent2)
        offspring.append(child)

    # Mutate the cloned set
    mutated_models = [perform_mutation(model) for model in duplicated_models]

    # Propagate weights of top-performing models to mutated models
    propagated_models = []
    for parent, mutated_model in zip(selected_models, mutated_models):
        propagated_model = propagate_weights(parent, mutated_model)
        propagated_models.append(propagated_model)

    # Create new agents with reduced parameters
    reduced_models = [reduce_parameters(model) for model in selected_models]

    # Combine all models for the next generation
    population = selected_models + offspring + mutated_models + propagated_models + reduced_models

    # Check for termination condition
    best_model = ranked_models[0][0][0]
    if best_model['fitness'] >= threshold_accuracy:
        break

    # Calculate fitness function for the remaining models and log the metrics
    fitness_scores = []
    accuracies = []
    inference_times = []
    memory_footprints = []

    for model_list in population:
        model = model_list[0][0]
        fitness = calculate_fitness(model['accuracy'], model['inference_time'], model['memory_footprint'])
        fitness_scores.append(fitness)
        accuracies.append(model['accuracy'])
        inference_times.append(model['inference_time'])
        memory_footprints.append(model['memory_footprint'])

    # Select the model with the highest fitness score
    best_model_index = fitness_scores.index(max(fitness_scores))
    best_model = population[best_model_index][0][0]
    best_hyperparameters.append(best_model)

    # Log metrics to TensorBoard
    tensorboard_callback.on_epoch_end(generation, {'accuracy': accuracies, 'inference_time': inference_times, 'memory_footprint': memory_footprints})

Epoch 1/10
1250/1250 [==============================] - 53s 42ms/step - loss: 1.6635 - accuracy: 0.3996 - val_loss: 1.4109 - val_accuracy: 0.4943
Epoch 2/10
1250/1250 [==============================] - 50s 40ms/step - loss: 1.3204 - accuracy: 0.5329 - val_loss: 1.2605 - val_accuracy: 0.5652
Epoch 3/10
1250/1250 [==============================] - 51s 41ms/step - loss: 1.1965 - accuracy: 0.5798 - val_loss: 1.1663 - val_accuracy: 0.5923
Epoch 4/10
1250/1250 [==============================] - 49s 39ms/step - loss: 1.1174 - accuracy: 0.6101 - val_loss: 1.1012 - val_accuracy: 0.6171
Epoch 5/10
1250/1250 [==============================] - 52s 42ms/step - loss: 1.0631 - accuracy: 0.6304 - val_loss: 1.1102 - val_accuracy: 0.6093
Epoch 6/10
1250/1250 [==============================] - 50s 40ms/step - loss: 1.0146 - accuracy: 0.6458 - val_loss: 1.0353 - val_accuracy: 0.6388
Epoch 7/10
1250/1250 [==============================] - 50s 40ms/step - loss: 0.9776 - accuracy: 0.6614 - val_loss: 1.0087 -

KeyError: ignored

In [ ]:
# Visualize the evolution of accuracy, inference time, and memory footprint over iterations
iterations = range(len(accuracies))

plt.plot(iterations, accuracies, label='Accuracy')
plt.plot(iterations, inference_times, label='Inference Time')
plt.plot(iterations, memory_footprints, label='Memory Footprint')

plt.xlabel('Iterations')
plt.ylabel('Value')
plt.title('Evolution of Accuracy, Inference Time, and Memory Footprint')
plt.legend()
plt.show()

In [ ]:

# Plot for accuracy
plt.figure(figsize=(10, 6))
plt.plot(range(len(accuracies)), accuracies, marker='o')
plt.xlabel('Generation Number')
plt.ylabel('Accuracy')
plt.title('Accuracy over Generations')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(len(f1_scores)), f1_scores, marker='o', color='green')
plt.xlabel('Generation Number')
plt.ylabel('F1 Score')
plt.title('F1 Score over Generations')
plt.grid(True)
plt.show()

In [ ]:
# Plot for memory footprint
plt.figure(figsize=(10, 6))
plt.plot(range(len(memory_footprints)), memory_footprints, marker='o', color='orange')
plt.xlabel('Generation Number')
plt.ylabel('Memory Footprint (MB)')
plt.title('Memory Footprint over Generations')
plt.grid(True)
plt.show()

In [ ]:

plt.figure(figsize=(10, 6))
plt.plot(range(len(inference_times)), inference_times, marker='o', color='red')
plt.xlabel('Generation Number')
plt.ylabel('Inference time (min)')
plt.title('Inference time evolution over Generations')
plt.grid(True)
plt.show()